In [1]:
import random
import string
import time
import base64
from hashlib import sha256
from hmac import HMAC

import http.client
import requests
import pandas as pd 

APPID = "tubmafwrzhpgfiuf"
SECRET = "eotpcqbvhycdshwscqnytiwzbgonposs"


def generate_nonce(length=32):
    """Generate a random nonce."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


def get_timestamp():
    """Get the current timestamp."""
    return int(time.time())


def build_sign_str(appid, nonce, timestamp):
    """Build the string to be signed."""
    return f'appid={appid}&nonce={nonce}&timestamp={timestamp}'


def calculate_signature(secret, message):
    """Calculate the HMAC SHA-256 signature."""
    return base64.urlsafe_b64encode(HMAC(secret.encode('utf-8'), message.encode('utf-8'), sha256).digest()).decode('utf-8')


def fetch_indicator_details(indicator_id):
    """Fetch the details for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/data?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def fetch_indicator_name(indicator_id):
    """Fetch the name for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/detail?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data').get('EdbName')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def main():
    # List of indicator IDs you want to fetch
    indicator_ids = ['C2503121015162988','C2501061639001179','C2208093900','Cstl2503121037598536',
                     'C2401291217','S2707412','C2303105344','C2504141014337084','C22504132138352664','C2310108734','C2407111550074601','C2210120092',
                     'C2311216480_250214172554','C2306071557','C22508111711406832']  # Add more IDs as needed

    # Dictionary to store DataFrames for each indicator
    data_frames = {}

    for indicator_id in indicator_ids:
        data = fetch_indicator_details(indicator_id)
        if data:
            # Create a DataFrame with DataTime as index
            df = pd.DataFrame(data)
            df['DataTime'] = pd.to_datetime(df['DataTime'])
            df.set_index('DataTime', inplace=True)
            df.sort_index(inplace=True)
            # Only keep the 'Value' column and rename it to the indicator ID
            df = df[['Value']].rename(columns={'Value': fetch_indicator_name(indicator_id)})
            data_frames[indicator_id] = df

    # Concatenate all DataFrames along the columns
    if data_frames:
        result_df = pd.concat(data_frames.values(), axis=1)
        print(result_df.info())
        result_df.to_excel("data_input/玻璃期货价格拟合残差-玻璃企业库存.xlsx")
        print("Data saved successfully as '玻璃期货价格拟合残差-玻璃企业库存.xlsx'")



if __name__ == "__main__":
    main()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6319 entries, 2005-01-31 to 2025-12-26
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   玻璃期货价格拟合残差/玻璃企业库存                     1548 non-null   float64
 1   中国10年期国债收益率Non-Trend/F0.2             4340 non-null   float64
 2   玻璃现货基差（指数）                            1379 non-null   float64
 3   30大中城市商品房成交面积/7DMATrend/F0.05         5775 non-null   float64
 4   玻璃社会库存/沙河超季节性/3年                      135 non-null    float64
 5   70大中城市新房房价环比                          177 non-null    float64
 6   每日建材成交/66DMA同比                        2304 non-null   float64
 7   玻璃区域价差（华中-西南）                         1401 non-null   float64
 8   玻璃企业库存(日熔调整/表需季节性预测)1周环差超季节性/3年       232 non-null    float64
 9   新增人民币贷款/居民/中长期/3MMA                   218 non-null    float64
 10  玻璃区域价差（华东-华中）                         1401 non-null   float64
 11 

Data saved successfully as '玻璃期货价格拟合残差-玻璃企业库存.xlsx'
